# Cycling Comfort Analysis Script

## Import packages

In [1]:
import arcpy
import os

## Input Data Paths

Set your data_path as your data folder path and your out_path as your project GDB path

In [4]:
p = arcpy.mp.ArcGISProject('current')
map = p.listMaps('Map')[0]

data_path = r"C:\App Challenge 2026" 
out_path = r"C:\App Challenge 2026\ScriptTest5" #Set as your project GDB

## Add Bikeshare Stations

Using the XY table to point tool, add your city of Toronto bikeshare stations to the map. You will need to convert your JSON to a CSV file before conducting this step.

In [5]:
arcpy.management.XYTableToPoint(
    in_table=os.path.join(data_path, "Bikeshare_LatLong.csv"),
    out_feature_class=os.path.join(out_path, "Bikeshare_stations"),
    x_field="data__stations__lon",
    y_field="data__stations__lat",
    z_field=None,
    coordinate_system='GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]];-400 -400 1000000000;-100000 10000;-100000 10000;8.98315284119521E-09;0.001;0.001;IsHighPrecision'
)

<Result 'C:\\App Challenge 2026\\ScriptTest5\\Bikeshare_stations.shp'>

## Limit Centrelines to Road/Trail Network

Limit your centrelines to just roads, laneways, trails, etc. The City of Toronto has a lot of highways that are very illegal to bike on, so we do not want to have those accounted for in our analysis.

In [6]:
arcpy.management.MakeFeatureLayer(
    in_features = os.path.join(data_path, "Centreline - Version 2 - 4326.shp"), 
    out_layer = "Centrelines_Filtered",
    where_clause = "FEATURE36 IN ('Access Road', 'Laneway', 'Local', 'Major Arterial', 'Minor Arterial', 'Major Arterial Ramp', 'Minor Arterial Ramp', 'Other', 'Other Ramp', 'Walkway', 'Trail')"
)

<Result 'Centrelines_Filtered'>

## Create Streetlight Data

In [7]:
arcpy.management.MakeFeatureLayer(
    in_features = os.path.join(data_path, "Poles - 4326.shp"), 
    out_layer = "Streetlights",
    where_clause = "SUBTYPE3 IN ('Street Light Pole', 'Street Light and Traffic Signal Pole')"
)

<Result 'Streetlights'>

## Create Traffic Light Data

In [8]:
arcpy.management.MakeFeatureLayer(
    in_features = os.path.join(data_path, "Poles - 4326.shp"), 
    out_layer = "Traffic Lights",
    where_clause = "SUBTYPE3 IN ('Traffic Signal Pole', 'Street Light and Traffic Signal Pole')"
)

<Result 'Traffic Lights'>

## Spatial Join Bike Lane Information to Road/Trail Network

Using a search radius of 15 metres, join the cycling network dataset and attributes to the road network. This will help us classify which roads are safe to bike on and which ones are stressful for cyclists.

In [9]:
arcpy.analysis.SpatialJoin(
    target_features="Centrelines_Filtered",
    join_features=os.path.join(data_path, "cycling-network - 4326.shp"),
    out_feature_class=os.path.join(out_path, "Centrelines_SpatialJoin"),
    join_operation="JOIN_ONE_TO_ONE",
    join_type="KEEP_ALL",
    field_mapping=r'_id1 "_id1" true true false 18 Double 0 0,First,#,Centrelines_Filtered,_id1,-1,-1;CENTREL2 "CENTREL2" true true false 18 Double 0 0,First,#,Centrelines_Filtered,CENTREL2,-1,-1;LINEAR_3 "LINEAR_3" true true false 18 Double 0 0,First,#,Centrelines_Filtered,LINEAR_3,-1,-1;LINEAR_4 "LINEAR_4" true true false 80 Text 0 0,First,#,Centrelines_Filtered,LINEAR_4,0,79;LINEAR_5 "LINEAR_5" true true false 80 Text 0 0,First,#,Centrelines_Filtered,LINEAR_5,0,79;ADDRESS6 "ADDRESS6" true true false 80 Text 0 0,First,#,Centrelines_Filtered,ADDRESS6,0,79;ADDRESS7 "ADDRESS7" true true false 80 Text 0 0,First,#,Centrelines_Filtered,ADDRESS7,0,79;PARITY_8 "PARITY_8" true true false 80 Text 0 0,First,#,Centrelines_Filtered,PARITY_8,0,79;PARITY_9 "PARITY_9" true true false 80 Text 0 0,First,#,Centrelines_Filtered,PARITY_9,0,79;LO_NUM_10 "LO_NUM_10" true true false 18 Double 0 0,First,#,Centrelines_Filtered,LO_NUM_10,-1,-1;HI_NUM_11 "HI_NUM_11" true true false 18 Double 0 0,First,#,Centrelines_Filtered,HI_NUM_11,-1,-1;LO_NUM_12 "LO_NUM_12" true true false 18 Double 0 0,First,#,Centrelines_Filtered,LO_NUM_12,-1,-1;HI_NUM_13 "HI_NUM_13" true true false 18 Double 0 0,First,#,Centrelines_Filtered,HI_NUM_13,-1,-1;BEGIN_A14 "BEGIN_A14" true true false 18 Double 0 0,First,#,Centrelines_Filtered,BEGIN_A14,-1,-1;END_ADD15 "END_ADD15" true true false 18 Double 0 0,First,#,Centrelines_Filtered,END_ADD15,-1,-1;BEGIN_A16 "BEGIN_A16" true true false 18 Double 0 0,First,#,Centrelines_Filtered,BEGIN_A16,-1,-1;END_ADD17 "END_ADD17" true true false 18 Double 0 0,First,#,Centrelines_Filtered,END_ADD17,-1,-1;BEGIN_A18 "BEGIN_A18" true true false 18 Double 0 0,First,#,Centrelines_Filtered,BEGIN_A18,-1,-1;END_ADD19 "END_ADD19" true true false 18 Double 0 0,First,#,Centrelines_Filtered,END_ADD19,-1,-1;BEGIN_A20 "BEGIN_A20" true true false 18 Double 0 0,First,#,Centrelines_Filtered,BEGIN_A20,-1,-1;END_ADD21 "END_ADD21" true true false 18 Double 0 0,First,#,Centrelines_Filtered,END_ADD21,-1,-1;LOW_NUM22 "LOW_NUM22" true true false 18 Double 0 0,First,#,Centrelines_Filtered,LOW_NUM22,-1,-1;HIGH_NU23 "HIGH_NU23" true true false 18 Double 0 0,First,#,Centrelines_Filtered,HIGH_NU23,-1,-1;LOW_NUM24 "LOW_NUM24" true true false 18 Double 0 0,First,#,Centrelines_Filtered,LOW_NUM24,-1,-1;HIGH_NU25 "HIGH_NU25" true true false 18 Double 0 0,First,#,Centrelines_Filtered,HIGH_NU25,-1,-1;LINEAR_26 "LINEAR_26" true true false 80 Text 0 0,First,#,Centrelines_Filtered,LINEAR_26,0,79;LINEAR_27 "LINEAR_27" true true false 80 Text 0 0,First,#,Centrelines_Filtered,LINEAR_27,0,79;LINEAR_28 "LINEAR_28" true true false 80 Text 0 0,First,#,Centrelines_Filtered,LINEAR_28,0,79;LINEAR_29 "LINEAR_29" true true false 80 Text 0 0,First,#,Centrelines_Filtered,LINEAR_29,0,79;LINEAR_30 "LINEAR_30" true true false 80 Text 0 0,First,#,Centrelines_Filtered,LINEAR_30,0,79;FROM_IN31 "FROM_IN31" true true false 18 Double 0 0,First,#,Centrelines_Filtered,FROM_IN31,-1,-1;TO_INTE32 "TO_INTE32" true true false 18 Double 0 0,First,#,Centrelines_Filtered,TO_INTE32,-1,-1;ONEWAY_33 "ONEWAY_33" true true false 18 Double 0 0,First,#,Centrelines_Filtered,ONEWAY_33,-1,-1;ONEWAY_34 "ONEWAY_34" true true false 80 Text 0 0,First,#,Centrelines_Filtered,ONEWAY_34,0,79;FEATURE35 "FEATURE35" true true false 18 Double 0 0,First,#,Centrelines_Filtered,FEATURE35,-1,-1;FEATURE36 "FEATURE36" true true false 80 Text 0 0,First,#,Centrelines_Filtered,FEATURE36,0,79;JURISDI37 "JURISDI37" true true false 80 Text 0 0,First,#,Centrelines_Filtered,JURISDI37,0,79;CENTREL38 "CENTREL38" true true false 80 Text 0 0,First,#,Centrelines_Filtered,CENTREL38,0,79;OBJECTI39 "OBJECTI39" true true false 18 Double 0 0,First,#,Centrelines_Filtered,OBJECTI39,-1,-1;MI_PRIN40 "MI_PRIN40" true true false 18 Double 0 0,First,#,Centrelines_Filtered,MI_PRIN40,-1,-1;_id1_1 "_id1" true true false 18 Double 0 0,First,#,Centrelines_Filtered,_id1,-1,-1;CENTREL2_1 "CENTREL2" true true false 18 Double 0 0,First,#,Centrelines_Filtered,CENTREL2,-1,-1;LINEAR_34 "LINEAR_3" true true false 18 Double 0 0,First,#,Centrelines_Filtered,LINEAR_3,-1,-1;LINEAR_45 "LINEAR_4" true true false 80 Text 0 0,First,#,Centrelines_Filtered,LINEAR_4,0,79;LINEAR_56 "LINEAR_5" true true false 80 Text 0 0,First,#,Centrelines_Filtered,LINEAR_5,0,79;ADDRESS6_1 "ADDRESS6" true true false 80 Text 0 0,First,#,Centrelines_Filtered,ADDRESS6,0,79;ADDRESS7_1 "ADDRESS7" true true false 80 Text 0 0,First,#,Centrelines_Filtered,ADDRESS7,0,79;PARITY_89 "PARITY_8" true true false 80 Text 0 0,First,#,Centrelines_Filtered,PARITY_8,0,79;PARITY__10 "PARITY_9" true true false 80 Text 0 0,First,#,Centrelines_Filtered,PARITY_9,0,79;LO_NUM__11 "LO_NUM_10" true true false 18 Double 0 0,First,#,Centrelines_Filtered,LO_NUM_10,-1,-1;HI_NUM__12 "HI_NUM_11" true true false 18 Double 0 0,First,#,Centrelines_Filtered,HI_NUM_11,-1,-1;LO_NUM__13 "LO_NUM_12" true true false 18 Double 0 0,First,#,Centrelines_Filtered,LO_NUM_12,-1,-1;HI_NUM__14 "HI_NUM_13" true true false 18 Double 0 0,First,#,Centrelines_Filtered,HI_NUM_13,-1,-1;BEGIN_A1_1 "BEGIN_A14" true true false 18 Double 0 0,First,#,Centrelines_Filtered,BEGIN_A14,-1,-1;END_ADD1_1 "END_ADD15" true true false 18 Double 0 0,First,#,Centrelines_Filtered,END_ADD15,-1,-1;BEGIN_A1_2 "BEGIN_A16" true true false 18 Double 0 0,First,#,Centrelines_Filtered,BEGIN_A16,-1,-1;END_ADD1_2 "END_ADD17" true true false 18 Double 0 0,First,#,Centrelines_Filtered,END_ADD17,-1,-1;BEGIN_A1_3 "BEGIN_A18" true true false 18 Double 0 0,First,#,Centrelines_Filtered,BEGIN_A18,-1,-1;END_ADD1_3 "END_ADD19" true true false 18 Double 0 0,First,#,Centrelines_Filtered,END_ADD19,-1,-1;BEGIN_A2_1 "BEGIN_A20" true true false 18 Double 0 0,First,#,Centrelines_Filtered,BEGIN_A20,-1,-1;END_ADD2_1 "END_ADD21" true true false 18 Double 0 0,First,#,Centrelines_Filtered,END_ADD21,-1,-1;LOW_NUM2_1 "LOW_NUM22" true true false 18 Double 0 0,First,#,Centrelines_Filtered,LOW_NUM22,-1,-1;HIGH_NU2_1 "HIGH_NU23" true true false 18 Double 0 0,First,#,Centrelines_Filtered,HIGH_NU23,-1,-1;LOW_NUM2_2 "LOW_NUM24" true true false 18 Double 0 0,First,#,Centrelines_Filtered,LOW_NUM24,-1,-1;HIGH_NU2_2 "HIGH_NU25" true true false 18 Double 0 0,First,#,Centrelines_Filtered,HIGH_NU25,-1,-1;LINEAR__27 "LINEAR_26" true true false 80 Text 0 0,First,#,Centrelines_Filtered,LINEAR_26,0,79;LINEAR__28 "LINEAR_27" true true false 80 Text 0 0,First,#,Centrelines_Filtered,LINEAR_27,0,79;LINEAR__29 "LINEAR_28" true true false 80 Text 0 0,First,#,Centrelines_Filtered,LINEAR_28,0,79;LINEAR__30 "LINEAR_29" true true false 80 Text 0 0,First,#,Centrelines_Filtered,LINEAR_29,0,79;LINEAR__31 "LINEAR_30" true true false 80 Text 0 0,First,#,Centrelines_Filtered,LINEAR_30,0,79;FROM_IN3_1 "FROM_IN31" true true false 18 Double 0 0,First,#,Centrelines_Filtered,FROM_IN31,-1,-1;TO_INTE3_1 "TO_INTE32" true true false 18 Double 0 0,First,#,Centrelines_Filtered,TO_INTE32,-1,-1;ONEWAY__34 "ONEWAY_33" true true false 18 Double 0 0,First,#,Centrelines_Filtered,ONEWAY_33,-1,-1;ONEWAY__35 "ONEWAY_34" true true false 80 Text 0 0,First,#,Centrelines_Filtered,ONEWAY_34,0,79;FEATURE3_1 "FEATURE35" true true false 18 Double 0 0,First,#,Centrelines_Filtered,FEATURE35,-1,-1;FEATURE3_2 "FEATURE36" true true false 80 Text 0 0,First,#,Centrelines_Filtered,FEATURE36,0,79;JURISDI3_1 "JURISDI37" true true false 80 Text 0 0,First,#,Centrelines_Filtered,JURISDI37,0,79;CENTREL3_1 "CENTREL38" true true false 80 Text 0 0,First,#,Centrelines_Filtered,CENTREL38,0,79;OBJECTI3_1 "OBJECTI39" true true false 18 Double 0 0,First,#,Centrelines_Filtered,OBJECTI39,-1,-1;MI_PRIN4_1 "MI_PRIN40" true true false 18 Double 0 0,First,#,Centrelines_Filtered,MI_PRIN40,-1,-1;CNPCLAS10_First "CNPCLAS10_First" true true false 255 Text 0 0,First,#,C:\App Challenge 2026\cycling-network - 4326.shp,CNPCLAS10,0,79;CONVERT16_First "CONVERT16_First" true true false 255 Text 0 0,First,#,C:\App Challenge 2026\cycling-network - 4326.shp,CONVERT16,0,79;DIR_LOW13_First "DIR_LOW13_First" true true false 255 Text 0 0,First,#,C:\App Challenge 2026\cycling-network - 4326.shp,DIR_LOW13,0,79;FID_First "FID_First" true true false 255 Long 0 0,First,#,C:\App Challenge 2026\cycling-network - 4326.shp,FID,-1,-1;FROM_ST7_First "FROM_ST7_First" true true false 255 Text 0 0,First,#,C:\App Challenge 2026\cycling-network - 4326.shp,FROM_ST7,0,79;INFRA_H15_First "INFRA_H15_First" true true false 255 Text 0 0,First,#,C:\App Challenge 2026\cycling-network - 4326.shp,INFRA_H15,0,79;INFRA_L14_First "INFRA_L14_First" true true false 255 Text 0 0,First,#,C:\App Challenge 2026\cycling-network - 4326.shp,INFRA_L14,0,79;INSTALL3_First "INSTALL3_First" true true false 255 Double 0 0,First,#,C:\App Challenge 2026\cycling-network - 4326.shp,INSTALL3,-1,-1;OWNER12_First "OWNER12_First" true true false 255 Text 0 0,First,#,C:\App Challenge 2026\cycling-network - 4326.shp,OWNER12,0,79;PRE_AMA5_First "PRE_AMA5_First" true true false 255 Text 0 0,First,#,C:\App Challenge 2026\cycling-network - 4326.shp,PRE_AMA5,0,79;ROADCLA9_First "ROADCLA9_First" true true false 255 Text 0 0,First,#,C:\App Challenge 2026\cycling-network - 4326.shp,ROADCLA9,0,79;SEGMENT2_First "SEGMENT2_First" true true false 255 Double 0 0,First,#,C:\App Challenge 2026\cycling-network - 4326.shp,SEGMENT2,-1,-1;STREET_6_First "STREET_6_First" true true false 255 Text 0 0,First,#,C:\App Challenge 2026\cycling-network - 4326.shp,STREET_6,0,79;SURFACE11_First "SURFACE11_First" true true false 255 Text 0 0,First,#,C:\App Challenge 2026\cycling-network - 4326.shp,SURFACE11,0,79;TO_STRE8_First "TO_STRE8_First" true true false 255 Text 0 0,First,#,C:\App Challenge 2026\cycling-network - 4326.shp,TO_STRE8,0,79;UPGRADE4_First "UPGRADE4_First" true true false 255 Double 0 0,First,#,C:\App Challenge 2026\cycling-network - 4326.shp,UPGRADE4,-1,-1',
    match_option="WITHIN_A_DISTANCE",
    search_radius="15 Meters",
    distance_field_name="",
    match_fields="STREET_6 LINEAR_4"
)

<Result 'C:\\App Challenge 2026\\ScriptTest5\\Centrelines_SpatialJoin.shp'>

## Create Roadway Buffers

Buffer the roadways. This helps keep the raster cells on bounded to the roadways when we generate them.

In [10]:
arcpy.analysis.Buffer(
    in_features="Centrelines_SpatialJoin",
    out_feature_class=os.path.join(out_path, "Centrelines_Buffer"),
    buffer_distance_or_field="10 Meters",
    line_side="FULL",
    line_end_type="ROUND",
    dissolve_option="ALL",
    dissolve_field=None,
    method="PLANAR"
)

<Result 'C:\\App Challenge 2026\\ScriptTest5\\Centrelines_Buffer.shp'>

## Create LTS Classification

LTS (Level of Traffic Stress) is a useful metric for planners looking to see how comfortable pedestrians and cyclists will be utilizing certain infrastructure. Our team has come up with a classification scheme here, however LTS is variable depending on the comfort level of the cyclist.

In [11]:
arcpy.management.CalculateField(
    in_table="Centrelines_SpatialJoin",
    field="LTS",
    expression="calculate_lst(!FEATURE3_2!, !INFRA_H15_!)",
    expression_type="PYTHON3",
    code_block="""def calculate_lst(road, bike):
    if road in ['Trail', 'Laneway', 'Walkway'] or bike in ['Park Road', 'Multi-Use Trail', 'Multi-Use Trail - Boulevard', 'Multi-Use Trail - Connector', 'Multi-Use Trail - Entrance', 'Multi-Use Trail - Existing Connector']:
        return 1
    
    # LTS 2
    elif bike in ['Bi-Directional Cycle Track', 'Cycle Track', 'Cycle Track - Contraflow', 'Bike Lane - Buffered'] or road == 'Local':
        return 2
    
    # LTS 3
    elif road in ['Minor Arterial', 'Minor Arterial Ramp', 'Access Road'] and bike in ['Bike Lane', 'Bike Lane - Contraflow']:
        return 3
        
    # LTS 4
    elif (road in ['Minor Arterial', 'Minor Arterial Ramp', 'Access Road'] and bike in ['Sharrows', 'Sharrows - Arterial', 'Sharrows - Arterial - Connector', 'Sharrows - Wayfinding', 'Signed Route (No Pavement Markings)', None]) or (road in ['Major Arterial', 'Major Arterial Ramp'] and bike in ['Bike Lane', 'Bike Lane - Contraflow']):
        return 4
        
    # LTS 5
    elif road in ['Major Arterial', 'Major Arterial Ramp'] and bike in ['Sharrows', 'Sharrows - Arterial', 'Sharrows - Arterial - Connector', 'Sharrows - Wayfinding', 'Signed Route (No Pavement Markings)', None]:
        return 5
    
    return 0""",
    field_type="SHORT",
    enforce_domains="NO_ENFORCE_DOMAINS"
)

<Result 'Centrelines_SpatialJoin'>

## Create Theft Density Raster

Create a raster for the density of bike thefts throughout the city of Toronto. Cell size and bandwidth for any of the following rasters can be changed to see how results differ.

In [12]:
with arcpy.EnvManager(outputCoordinateSystem=None, extent='-79.6392568669108 43.5822069225115 -79.118033639241 43.855454343843 GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]', mask="Centrelines_Buffer", scratchWorkspace=r"C:\App Challenge 2026\ScriptTest App Challenge 2026\ScriptTest App Challenge 2026.gdb"):
    BikeTheftDensity = arcpy.sa.KernelDensity(
        in_features=os.path.join(data_path, "BIKETHEFT_OPEN_DATA.shp"),
        population_field="NONE",
        cell_size=25,
        search_radius=300,
        area_unit_scale_factor="SQUARE_METERS",
        out_cell_values="DENSITIES",
        method="PLANAR",
        in_barriers=None
    )

## Create Accident Density Raster

Create a raster for the density of bike accidents throughout the city of Toronto. Cell size and bandwidth for any of the following rasters can be changed to see how results differ.

In [13]:
with arcpy.EnvManager(outputCoordinateSystem='PROJCS["WGS_1984_Web_Mercator_Auxiliary_Sphere",GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Mercator_Auxiliary_Sphere"],PARAMETER["False_Easting",0.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",0.0],PARAMETER["Standard_Parallel_1",0.0],PARAMETER["Auxiliary_Sphere_Type",0.0],UNIT["Meter",1.0]]', snapRaster="BikeTheftDensity", extent='-79.6392568669108 43.5822069225115 -79.1180094233005 43.8553776414644 GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]', cellSize="BikeTheftDensity", mask="BikeTheftDensity", scratchWorkspace=out_path):
    BikeCollisions = arcpy.sa.KernelDensity(
        in_features=os.path.join(data_path, "CYCLIST_KSI.shp"),
        population_field="NONE",
        cell_size=25,
        search_radius=300,
        area_unit_scale_factor="SQUARE_METERS",
        out_cell_values="DENSITIES",
        method="PLANAR",
        in_barriers=None
    )

## Create Bike Rack Density Raster

Create a raster for the density of bike racks throughout the City of Toronto. Cell size and bandwidth for any of the following rasters can be changed to see how results differ.

In [14]:
with arcpy.EnvManager(outputCoordinateSystem='PROJCS["WGS_1984_Web_Mercator_Auxiliary_Sphere",GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Mercator_Auxiliary_Sphere"],PARAMETER["False_Easting",0.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",0.0],PARAMETER["Standard_Parallel_1",0.0],PARAMETER["Auxiliary_Sphere_Type",0.0],UNIT["Meter",1.0]]', snapRaster="BikeTheftDensity", extent='-79.6392568669108 43.5822069225115 -79.1180094233005 43.8553776414644 GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]', cellSize="BikeTheftDensity", mask="BikeTheftDensity", scratchWorkspace=r"C:\App Challenge 2026\ScriptTest App Challenge 2026\ScriptTest App Challenge 2026.gdb"):
    BikeParkingDensity = arcpy.sa.KernelDensity(
        in_features=os.path.join(data_path, "Bicycle Parking Map Data - 4326.shp"),
        population_field="NONE",
        cell_size=25,
        search_radius=300,
        area_unit_scale_factor="SQUARE_METERS",
        out_cell_values="DENSITIES",
        method="PLANAR",
        in_barriers=None
    )

## Create Bikeshare Density Raster

Create a raster for the density of bikeshare stations throughout the City of Toronto. Cell size and bandwidth for any of the following rasters can be changed to see how results differ.

In [15]:
with arcpy.EnvManager(outputCoordinateSystem='PROJCS["WGS_1984_Web_Mercator_Auxiliary_Sphere",GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Mercator_Auxiliary_Sphere"],PARAMETER["False_Easting",0.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",0.0],PARAMETER["Standard_Parallel_1",0.0],PARAMETER["Auxiliary_Sphere_Type",0.0],UNIT["Meter",1.0]]', snapRaster="BikeTheftDensity", extent='-79.6392568669108 43.5822069225115 -79.1180094233005 43.8553776414644 GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]', cellSize="BikeTheftDensity", mask="BikeTheftDensity", scratchWorkspace=out_path):
    BikeshareDensity = arcpy.sa.KernelDensity(
        in_features="Bikeshare_stations",
        population_field="NONE",
        cell_size=25,
        search_radius=300,
        area_unit_scale_factor="SQUARE_METERS",
        out_cell_values="DENSITIES",
        method="PLANAR",
        in_barriers=None
    )

## Create Streetlight Density Raster

Create a raster for the density of bikeshare stations throughout the City of Toronto. Cell size and bandwidth for any of the following rasters can be changed to see how results differ.

In [16]:
with arcpy.EnvManager(outputCoordinateSystem='PROJCS["WGS_1984_Web_Mercator_Auxiliary_Sphere",GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Mercator_Auxiliary_Sphere"],PARAMETER["False_Easting",0.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",0.0],PARAMETER["Standard_Parallel_1",0.0],PARAMETER["Auxiliary_Sphere_Type",0.0],UNIT["Meter",1.0]]', snapRaster="BikeTheftDensity", extent='-79.6392568669108 43.5822069225115 -79.1180094233005 43.8553776414644 GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]', cellSize="BikeTheftDensity", mask="BikeTheftDensity", scratchWorkspace=out_path):
    StreetlightDensity = arcpy.sa.KernelDensity(
        in_features="Streetlights",
        population_field="NONE",
        cell_size=25,
        search_radius=300,
        area_unit_scale_factor="SQUARE_METERS",
        out_cell_values="DENSITIES",
        method="PLANAR",
        in_barriers=None
    )

## Create LTS Raster

Create a raster for the LTS classification outlined above.

In [17]:
with arcpy.EnvManager(outputCoordinateSystem='PROJCS["WGS_1984_Web_Mercator_Auxiliary_Sphere",GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Mercator_Auxiliary_Sphere"],PARAMETER["False_Easting",0.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",0.0],PARAMETER["Standard_Parallel_1",0.0],PARAMETER["Auxiliary_Sphere_Type",0.0],UNIT["Meter",1.0]]', snapRaster="BikeTheftDensity", extent='-79.6392568669108 43.5822069225115 -79.1180094233005 43.8553776414644 GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]', cellSize="BikeTheftDensity"):
    arcpy.conversion.PolylineToRaster(
        in_features="Centrelines_SpatialJoin",
        value_field="LTS",
        out_rasterdataset=os.path.join(out_path, "BikeSafeIdx"),
        cell_assignment="MAXIMUM_LENGTH",
        priority_field="NONE",
        cellsize="BikeTheftDensity",
        build_rat="BUILD"
    )

## Reclassify All Rasters from 1-5

This block of code ensures that all rasters are on the same scale and are thus fit for weighted overlaying/summation.

In [18]:
## Bike Theft Density

with arcpy.EnvManager(scratchWorkspace=out_path):
    BikeTheftIndex = arcpy.sa.Reclassify(
        in_raster="BikeTheftDensity",
        reclass_field="VALUE",
        remap="0 0.000003 1;0.000003 0.000019 2;0.000019 0.000115 3;0.000115 0.000695 4;0.000695 0.004200 5",
        missing_values="DATA"
    )

## Bike Collision Density

with arcpy.EnvManager(scratchWorkspace=out_path):
    BikeCollisionIndex = arcpy.sa.Reclassify(
        in_raster="BikeCollisions",
        reclass_field="VALUE",
        remap="0 0.000001 1;0.000001 0.000004 2;0.000004 0.000018 3;0.000018 0.000068 4;0.000068 0.000265 5",
        missing_values="DATA"
    )

## Bike Parking Density

with arcpy.EnvManager(scratchWorkspace=out_path):
    BikeParkingIndex = arcpy.sa.Reclassify(
        in_raster="BikeParkingDensity",
        reclass_field="VALUE",
        remap="0 5;0 0.000001 4;0.000001 0.000003 3;0.000003 0.000012 2;0.000012 0.000047 1",
        missing_values="DATA"
    )

## Bikeshare Density

with arcpy.EnvManager(scratchWorkspace=out_path):
    BikeshareIndex = arcpy.sa.Reclassify(
        in_raster="BikeshareDensity",
        reclass_field="VALUE",
        remap="0 5;0 0.000001 4;0.000001 0.000004 3;0.000004 0.000015 2;0.000015 0.000059 1",
        missing_values="DATA"
    )

## Streetlight Density

with arcpy.EnvManager(scratchWorkspace=out_path):
    StreetlightIndex = arcpy.sa.Reclassify(
        in_raster="StreetlightDensity",
        reclass_field="VALUE",
        remap="0 0.000109 5;0.000109 0.000162 4;0.000162 0.000272 3;0.000272 0.000496 2;0.000496 0.000957 1",
        missing_values="DATA"
    )

## Biking Safety Index

with arcpy.EnvManager(scratchWorkspace=out_path):
    BikingSafetyIndexReclass = arcpy.sa.Reclassify(
        in_raster="BikeSafeIdx",
        reclass_field="Value",
        remap="0 1 1;1 2 2;2 3 3;3 4 4;4 5 5",
        missing_values="DATA"
    )

## Sum Rasters

Sum the outlined rasters to come up with a safety index and an accessibility index.

In [19]:
with arcpy.EnvManager(scratchWorkspace=out_path):
    SafetyIndexSum = arcpy.ia.CellStatistics(
        in_rasters_or_constants="BikingSafetyIndexReclass;BikeCollisionIndex;BikeTheftIndex;StreetlightIndex",
        statistics_type="SUM",
        ignore_nodata="DATA",
        process_as_multiband="SINGLE_BAND",
        percentile_value=90,
        percentile_interpolation_type="AUTO_DETECT"
    )

with arcpy.EnvManager(scratchWorkspace=out_path):
    AccessibilityIndexSum = arcpy.ia.CellStatistics(
        in_rasters_or_constants="BikeshareIndex;BikeParkingIndex",
        statistics_type="SUM",
        ignore_nodata="DATA",
        process_as_multiband="SINGLE_BAND",
        percentile_value=90,
        percentile_interpolation_type="AUTO_DETECT"
    )

## Overlay Safety and Accessibility Indices

Overlay the Safety and Accessibility indices. We have chosen weights of 33% to 67% as the accessibility index has 2 factors while the safety index has 4. Weights can be adjusted.

In [20]:
with arcpy.EnvManager(scratchWorkspace=out_path):
    WeightedCyclingIndex = arcpy.sa.WeightedOverlay(
        in_weighted_overlay_table="('AccessibilityIndexSum' 33 'Value' (1 1; 2 2; 3 3; 4 4; 5 6; 6 7; 7 8; 8 9; 9 10; 10 1; NODATA NODATA); 'SafetyIndexSum' 67 'Value' (1 1; 2 2; 3 3; 4 4; 5 5; 6 6; 7 7; 8 8; 9 9; 10 10; 11 1; 12 1; 13 1; 14 1; 15 1; NODATA NODATA));1 10 1"
    )

## Add Surface Information to Centrelines

Add surface information from our underlying raster cells (for all reclassed rasters generated) to our road and trail network. This tool uses the mean value of the raster cells underlying each road/trail section, but road/trail sections are short enough that data variation loss is minimal.

In [21]:
import arcpy

surfaces_to_process = {
    "WeightedCyclingIndex": "WghtCycMn",   
    "SafetyIndexSum": "SafeIdxMn",
    "AccessibilityIndexSum": "AccIdxMn",
    "BikeshareIndex": "BkShrIdxMn",
    "BikeParkingIndex": "BkPrkIdxMn",
    "BikeCollisionIndex": "BkColIdxMn",
    "BikeTheftIndex": "BkThfIdxMn",
    "StreetlightIndex": "StLtIdxMn",
    "BikingSafetyIndexReclass": "BkSafIdxMn",
}

feature_class = "Centrelines_SpatialJoin"

for surface, custom_name in surfaces_to_process.items():
    surface_path = surface if surface.endswith(".tif") else f"{surface}.tif"
    
    print(f"Processing surface: {surface_path}...")

    arcpy.sa.AddSurfaceInformation(
        in_feature_class=feature_class,
        in_surface=surface_path,
        out_property="Z_MEAN",
        method="BILINEAR"
    )
    
    short_name = custom_name[:10]
    print(f"  Creating field: {short_name}")
    
    arcpy.management.AddField(
        in_table=feature_class, 
        field_name=short_name, 
        field_type="DOUBLE",
        field_alias=custom_name.replace("_", " ")
    )
    
    arcpy.management.CalculateField(
        in_table=feature_class, 
        field=short_name, 
        expression="!Z_Mean!", 
        expression_type="PYTHON3"
    )
    
    arcpy.management.DeleteField(feature_class, "Z_Mean")

print("All surfaces processed successfully.")

Processing surface: WeightedCyclingIndex.tif...
  Creating field: WghtCycMn
Processing surface: SafetyIndexSum.tif...
  Creating field: SafeIdxMn
Processing surface: AccessibilityIndexSum.tif...
  Creating field: AccIdxMn
Processing surface: BikeshareIndex.tif...
  Creating field: BkShrIdxMn
Processing surface: BikeParkingIndex.tif...
  Creating field: BkPrkIdxMn
Processing surface: BikeCollisionIndex.tif...
  Creating field: BkColIdxMn
Processing surface: BikeTheftIndex.tif...
  Creating field: BkThfIdxMn
Processing surface: StreetlightIndex.tif...
  Creating field: StLtIdxMn
Processing surface: BikingSafetyIndexReclass.tif...
  Creating field: BkSafIdxMn
All surfaces processed successfully.


## Find Which Intersections Have Traffic Lights

This helps us measure impedance to travel time on a bicycle.

In [22]:
arcpy.analysis.Intersect(
    in_features="Centrelines_SpatialJoin",
    out_feature_class=os.path.join(out_path, "Intersections"),
    join_attributes="ALL",
    cluster_tolerance=None,
    output_type="POINT"
)

<Result 'C:\\App Challenge 2026\\ScriptTest5\\Intersections.shp'>

In [23]:
arcpy.management.SelectLayerByLocation(
    "Intersections", 
    "WITHIN_A_DISTANCE", 
    "Traffic Lights", 
    "30 Meters"
)

# 3. Create a new Feature Class from the selection
arcpy.management.CopyFeatures("Intersections", "Traffic_Lights")

<Result 'C:\\App Challenge 2026\\ScriptTest5\\ScriptTest5.gdb\\Traffic_Lights'>

## Spatial Join Traffic Light Count to Centrelines Layer

In [24]:
arcpy.analysis.SpatialJoin(
    target_features="Centrelines_SpatialJoin",
    join_features="Traffic_Lights",
    out_feature_class=os.path.join(out_path, "BikeCentrelines"),
    join_operation="JOIN_ONE_TO_ONE",
    join_type="KEEP_ALL",
    field_mapping='Light_Count "Light_Count" true true false 10 Long 0 0,First,#,Centrelines_SpatialJoin,Join_Count,-1,-1;TARGET_FID "TARGET_FID" true true false 10 Long 0 0,First,#,Centrelines_SpatialJoin,TARGET_FID,-1,-1;_id1 "_id1" true true false 19 Double 0 0,First,#,Centrelines_SpatialJoin,_id1,-1,-1;CENTREL2 "CENTREL2" true true false 19 Double 0 0,First,#,Centrelines_SpatialJoin,CENTREL2,-1,-1;LINEAR_3 "LINEAR_3" true true false 19 Double 0 0,First,#,Centrelines_SpatialJoin,LINEAR_3,-1,-1;LINEAR_4 "LINEAR_4" true true false 80 Text 0 0,First,#,Centrelines_SpatialJoin,LINEAR_4,0,79;LINEAR_5 "LINEAR_5" true true false 80 Text 0 0,First,#,Centrelines_SpatialJoin,LINEAR_5,0,79;ADDRESS6 "ADDRESS6" true true false 80 Text 0 0,First,#,Centrelines_SpatialJoin,ADDRESS6,0,79;ADDRESS7 "ADDRESS7" true true false 80 Text 0 0,First,#,Centrelines_SpatialJoin,ADDRESS7,0,79;PARITY_8 "PARITY_8" true true false 80 Text 0 0,First,#,Centrelines_SpatialJoin,PARITY_8,0,79;PARITY_9 "PARITY_9" true true false 80 Text 0 0,First,#,Centrelines_SpatialJoin,PARITY_9,0,79;LO_NUM_10 "LO_NUM_10" true true false 19 Double 0 0,First,#,Centrelines_SpatialJoin,LO_NUM_10,-1,-1;HI_NUM_11 "HI_NUM_11" true true false 19 Double 0 0,First,#,Centrelines_SpatialJoin,HI_NUM_11,-1,-1;LO_NUM_12 "LO_NUM_12" true true false 19 Double 0 0,First,#,Centrelines_SpatialJoin,LO_NUM_12,-1,-1;HI_NUM_13 "HI_NUM_13" true true false 19 Double 0 0,First,#,Centrelines_SpatialJoin,HI_NUM_13,-1,-1;BEGIN_A14 "BEGIN_A14" true true false 19 Double 0 0,First,#,Centrelines_SpatialJoin,BEGIN_A14,-1,-1;END_ADD15 "END_ADD15" true true false 19 Double 0 0,First,#,Centrelines_SpatialJoin,END_ADD15,-1,-1;BEGIN_A16 "BEGIN_A16" true true false 19 Double 0 0,First,#,Centrelines_SpatialJoin,BEGIN_A16,-1,-1;END_ADD17 "END_ADD17" true true false 19 Double 0 0,First,#,Centrelines_SpatialJoin,END_ADD17,-1,-1;BEGIN_A18 "BEGIN_A18" true true false 19 Double 0 0,First,#,Centrelines_SpatialJoin,BEGIN_A18,-1,-1;END_ADD19 "END_ADD19" true true false 19 Double 0 0,First,#,Centrelines_SpatialJoin,END_ADD19,-1,-1;BEGIN_A20 "BEGIN_A20" true true false 19 Double 0 0,First,#,Centrelines_SpatialJoin,BEGIN_A20,-1,-1;END_ADD21 "END_ADD21" true true false 19 Double 0 0,First,#,Centrelines_SpatialJoin,END_ADD21,-1,-1;LOW_NUM22 "LOW_NUM22" true true false 19 Double 0 0,First,#,Centrelines_SpatialJoin,LOW_NUM22,-1,-1;HIGH_NU23 "HIGH_NU23" true true false 19 Double 0 0,First,#,Centrelines_SpatialJoin,HIGH_NU23,-1,-1;LOW_NUM24 "LOW_NUM24" true true false 19 Double 0 0,First,#,Centrelines_SpatialJoin,LOW_NUM24,-1,-1;HIGH_NU25 "HIGH_NU25" true true false 19 Double 0 0,First,#,Centrelines_SpatialJoin,HIGH_NU25,-1,-1;LINEAR_26 "LINEAR_26" true true false 80 Text 0 0,First,#,Centrelines_SpatialJoin,LINEAR_26,0,79;LINEAR_27 "LINEAR_27" true true false 80 Text 0 0,First,#,Centrelines_SpatialJoin,LINEAR_27,0,79;LINEAR_28 "LINEAR_28" true true false 80 Text 0 0,First,#,Centrelines_SpatialJoin,LINEAR_28,0,79;LINEAR_29 "LINEAR_29" true true false 80 Text 0 0,First,#,Centrelines_SpatialJoin,LINEAR_29,0,79;LINEAR_30 "LINEAR_30" true true false 80 Text 0 0,First,#,Centrelines_SpatialJoin,LINEAR_30,0,79;FROM_IN31 "FROM_IN31" true true false 19 Double 0 0,First,#,Centrelines_SpatialJoin,FROM_IN31,-1,-1;TO_INTE32 "TO_INTE32" true true false 19 Double 0 0,First,#,Centrelines_SpatialJoin,TO_INTE32,-1,-1;ONEWAY_33 "ONEWAY_33" true true false 19 Double 0 0,First,#,Centrelines_SpatialJoin,ONEWAY_33,-1,-1;ONEWAY_34 "ONEWAY_34" true true false 80 Text 0 0,First,#,Centrelines_SpatialJoin,ONEWAY_34,0,79;FEATURE35 "FEATURE35" true true false 19 Double 0 0,First,#,Centrelines_SpatialJoin,FEATURE35,-1,-1;FEATURE36 "FEATURE36" true true false 80 Text 0 0,First,#,Centrelines_SpatialJoin,FEATURE36,0,79;JURISDI37 "JURISDI37" true true false 80 Text 0 0,First,#,Centrelines_SpatialJoin,JURISDI37,0,79;CENTREL38 "CENTREL38" true true false 80 Text 0 0,First,#,Centrelines_SpatialJoin,CENTREL38,0,79;OBJECTI39 "OBJECTI39" true true false 19 Double 0 0,First,#,Centrelines_SpatialJoin,OBJECTI39,-1,-1;MI_PRIN40 "MI_PRIN40" true true false 19 Double 0 0,First,#,Centrelines_SpatialJoin,MI_PRIN40,-1,-1;_id1_1 "_id1_1" true true false 19 Double 0 0,First,#,Centrelines_SpatialJoin,_id1_1,-1,-1;CENTREL2_1 "CENTREL2_1" true true false 19 Double 0 0,First,#,Centrelines_SpatialJoin,CENTREL2_1,-1,-1;LINEAR_34 "LINEAR_34" true true false 19 Double 0 0,First,#,Centrelines_SpatialJoin,LINEAR_34,-1,-1;LINEAR_45 "LINEAR_45" true true false 80 Text 0 0,First,#,Centrelines_SpatialJoin,LINEAR_45,0,79;LINEAR_56 "LINEAR_56" true true false 80 Text 0 0,First,#,Centrelines_SpatialJoin,LINEAR_56,0,79;ADDRESS6_1 "ADDRESS6_1" true true false 80 Text 0 0,First,#,Centrelines_SpatialJoin,ADDRESS6_1,0,79;ADDRESS7_1 "ADDRESS7_1" true true false 80 Text 0 0,First,#,Centrelines_SpatialJoin,ADDRESS7_1,0,79;PARITY_89 "PARITY_89" true true false 80 Text 0 0,First,#,Centrelines_SpatialJoin,PARITY_89,0,79;PARITY__10 "PARITY__10" true true false 80 Text 0 0,First,#,Centrelines_SpatialJoin,PARITY__10,0,79;LO_NUM__11 "LO_NUM__11" true true false 19 Double 0 0,First,#,Centrelines_SpatialJoin,LO_NUM__11,-1,-1;HI_NUM__12 "HI_NUM__12" true true false 19 Double 0 0,First,#,Centrelines_SpatialJoin,HI_NUM__12,-1,-1;LO_NUM__13 "LO_NUM__13" true true false 19 Double 0 0,First,#,Centrelines_SpatialJoin,LO_NUM__13,-1,-1;HI_NUM__14 "HI_NUM__14" true true false 19 Double 0 0,First,#,Centrelines_SpatialJoin,HI_NUM__14,-1,-1;BEGIN_A1_1 "BEGIN_A1_1" true true false 19 Double 0 0,First,#,Centrelines_SpatialJoin,BEGIN_A1_1,-1,-1;END_ADD1_1 "END_ADD1_1" true true false 19 Double 0 0,First,#,Centrelines_SpatialJoin,END_ADD1_1,-1,-1;BEGIN_A1_2 "BEGIN_A1_2" true true false 19 Double 0 0,First,#,Centrelines_SpatialJoin,BEGIN_A1_2,-1,-1;END_ADD1_2 "END_ADD1_2" true true false 19 Double 0 0,First,#,Centrelines_SpatialJoin,END_ADD1_2,-1,-1;BEGIN_A1_3 "BEGIN_A1_3" true true false 19 Double 0 0,First,#,Centrelines_SpatialJoin,BEGIN_A1_3,-1,-1;END_ADD1_3 "END_ADD1_3" true true false 19 Double 0 0,First,#,Centrelines_SpatialJoin,END_ADD1_3,-1,-1;BEGIN_A2_1 "BEGIN_A2_1" true true false 19 Double 0 0,First,#,Centrelines_SpatialJoin,BEGIN_A2_1,-1,-1;END_ADD2_1 "END_ADD2_1" true true false 19 Double 0 0,First,#,Centrelines_SpatialJoin,END_ADD2_1,-1,-1;LOW_NUM2_1 "LOW_NUM2_1" true true false 19 Double 0 0,First,#,Centrelines_SpatialJoin,LOW_NUM2_1,-1,-1;HIGH_NU2_1 "HIGH_NU2_1" true true false 19 Double 0 0,First,#,Centrelines_SpatialJoin,HIGH_NU2_1,-1,-1;LOW_NUM2_2 "LOW_NUM2_2" true true false 19 Double 0 0,First,#,Centrelines_SpatialJoin,LOW_NUM2_2,-1,-1;HIGH_NU2_2 "HIGH_NU2_2" true true false 19 Double 0 0,First,#,Centrelines_SpatialJoin,HIGH_NU2_2,-1,-1;LINEAR__27 "LINEAR__27" true true false 80 Text 0 0,First,#,Centrelines_SpatialJoin,LINEAR__27,0,79;LINEAR__28 "LINEAR__28" true true false 80 Text 0 0,First,#,Centrelines_SpatialJoin,LINEAR__28,0,79;LINEAR__29 "LINEAR__29" true true false 80 Text 0 0,First,#,Centrelines_SpatialJoin,LINEAR__29,0,79;LINEAR__30 "LINEAR__30" true true false 80 Text 0 0,First,#,Centrelines_SpatialJoin,LINEAR__30,0,79;LINEAR__31 "LINEAR__31" true true false 80 Text 0 0,First,#,Centrelines_SpatialJoin,LINEAR__31,0,79;FROM_IN3_1 "FROM_IN3_1" true true false 19 Double 0 0,First,#,Centrelines_SpatialJoin,FROM_IN3_1,-1,-1;TO_INTE3_1 "TO_INTE3_1" true true false 19 Double 0 0,First,#,Centrelines_SpatialJoin,TO_INTE3_1,-1,-1;ONEWAY__34 "ONEWAY__34" true true false 19 Double 0 0,First,#,Centrelines_SpatialJoin,ONEWAY__34,-1,-1;ONEWAY__35 "ONEWAY__35" true true false 80 Text 0 0,First,#,Centrelines_SpatialJoin,ONEWAY__35,0,79;FEATURE3_1 "FEATURE3_1" true true false 19 Double 0 0,First,#,Centrelines_SpatialJoin,FEATURE3_1,-1,-1;FEATURE3_2 "FEATURE3_2" true true false 80 Text 0 0,First,#,Centrelines_SpatialJoin,FEATURE3_2,0,79;JURISDI3_1 "JURISDI3_1" true true false 80 Text 0 0,First,#,Centrelines_SpatialJoin,JURISDI3_1,0,79;CENTREL3_1 "CENTREL3_1" true true false 80 Text 0 0,First,#,Centrelines_SpatialJoin,CENTREL3_1,0,79;OBJECTI3_1 "OBJECTI3_1" true true false 19 Double 0 0,First,#,Centrelines_SpatialJoin,OBJECTI3_1,-1,-1;MI_PRIN4_1 "MI_PRIN4_1" true true false 19 Double 0 0,First,#,Centrelines_SpatialJoin,MI_PRIN4_1,-1,-1;CNPCLAS10_ "CNPCLAS10_" true true false 254 Text 0 0,First,#,Centrelines_SpatialJoin,CNPCLAS10_,0,253;CONVERT16_ "CONVERT16_" true true false 254 Text 0 0,First,#,Centrelines_SpatialJoin,CONVERT16_,0,253;DIR_LOW13_ "DIR_LOW13_" true true false 254 Text 0 0,First,#,Centrelines_SpatialJoin,DIR_LOW13_,0,253;FID_First "FID_First" true true false 10 Long 0 0,First,#,Centrelines_SpatialJoin,FID_First,-1,-1;FROM_ST7_F "FROM_ST7_F" true true false 254 Text 0 0,First,#,Centrelines_SpatialJoin,FROM_ST7_F,0,253;INFRA_H15_ "INFRA_H15_" true true false 254 Text 0 0,First,#,Centrelines_SpatialJoin,INFRA_H15_,0,253;INFRA_L14_ "INFRA_L14_" true true false 254 Text 0 0,First,#,Centrelines_SpatialJoin,INFRA_L14_,0,253;INSTALL3_F "INSTALL3_F" true true false 19 Double 0 0,First,#,Centrelines_SpatialJoin,INSTALL3_F,-1,-1;OWNER12_Fi "OWNER12_Fi" true true false 254 Text 0 0,First,#,Centrelines_SpatialJoin,OWNER12_Fi,0,253;PRE_AMA5_F "PRE_AMA5_F" true true false 254 Text 0 0,First,#,Centrelines_SpatialJoin,PRE_AMA5_F,0,253;ROADCLA9_F "ROADCLA9_F" true true false 254 Text 0 0,First,#,Centrelines_SpatialJoin,ROADCLA9_F,0,253;SEGMENT2_F "SEGMENT2_F" true true false 19 Double 0 0,First,#,Centrelines_SpatialJoin,SEGMENT2_F,-1,-1;STREET_6_F "STREET_6_F" true true false 254 Text 0 0,First,#,Centrelines_SpatialJoin,STREET_6_F,0,253;SURFACE11_ "SURFACE11_" true true false 254 Text 0 0,First,#,Centrelines_SpatialJoin,SURFACE11_,0,253;TO_STRE8_F "TO_STRE8_F" true true false 254 Text 0 0,First,#,Centrelines_SpatialJoin,TO_STRE8_F,0,253;UPGRADE4_F "UPGRADE4_F" true true false 19 Double 0 0,First,#,Centrelines_SpatialJoin,UPGRADE4_F,-1,-1;LTS "LTS" true true false 5 Long 0 0,First,#,Centrelines_SpatialJoin,LTS,-1,-1;WghtCycMn "WghtCycMn" true true false 19 Double 0 0,First,#,Centrelines_SpatialJoin,WghtCycMn,-1,-1;SafeIdxMn "SafeIdxMn" true true false 19 Double 0 0,First,#,Centrelines_SpatialJoin,SafeIdxMn,-1,-1;AccIdxMn "AccIdxMn" true true false 19 Double 0 0,First,#,Centrelines_SpatialJoin,AccIdxMn,-1,-1;BkShrIdxMn "BkShrIdxMn" true true false 19 Double 0 0,First,#,Centrelines_SpatialJoin,BkShrIdxMn,-1,-1;BkPrkIdxMn "BkPrkIdxMn" true true false 19 Double 0 0,First,#,Centrelines_SpatialJoin,BkPrkIdxMn,-1,-1;BkColIdxMn "BkColIdxMn" true true false 19 Double 0 0,First,#,Centrelines_SpatialJoin,BkColIdxMn,-1,-1;BkThfIdxMn "BkThfIdxMn" true true false 19 Double 0 0,First,#,Centrelines_SpatialJoin,BkThfIdxMn,-1,-1;StLtIdxMn "StLtIdxMn" true true false 19 Double 0 0,First,#,Centrelines_SpatialJoin,StLtIdxMn,-1,-1;BkSafIdxMn "BkSafIdxMn" true true false 19 Double 0 0,First,#,Centrelines_SpatialJoin,BkSafIdxMn,-1,-1',
    match_option="INTERSECT",
    search_radius=None,
    distance_field_name="",
    match_fields=None
)

<Result 'C:\\App Challenge 2026\\ScriptTest5\\BikeCentrelines.shp'>

## Calculate Bike Travel Time 

Travel Time is based on an 18 second wait at each light and an average speed of 15 km/h.

In [25]:
arcpy.management.CalculateGeometryAttributes(
    in_features="BikeCentrelines",
    geometry_property="Distance LENGTH_GEODESIC",
    length_unit="METERS",
    area_unit="",
    coordinate_system='GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]',
    coordinate_format="SAME_AS_INPUT"
)

<Result 'BikeCentrelines'>

In [28]:
arcpy.management.CalculateField(
    in_table="BikeCentrelines",
    field="Travel Time (Min)",
    expression="!Distance! / (15000/60) + !Light_Coun!*0.3",
    expression_type="PYTHON3",
    code_block="",
    field_type="TEXT",
    enforce_domains="NO_ENFORCE_DOMAINS"
)

<Result 'BikeCentrelines'>

## Clean up Centrelines Layer

Ensure the centrelines layer has only the necessary fields before uploading to AGOL.

In [29]:
arcpy.conversion.ExportFeatures(
    in_features="BikeCentrelines",
    out_features=os.path.join(out_path, "BikeCentrelines_Clean"),
    where_clause="",
    use_field_alias_as_name="NOT_USE_ALIAS",
    field_mapping=r'LTS "LTS" true true false 4 Long 0 0,First,#,BikeCentrelines,LTS,-1,-1;WghtCycMn "WghtCycMn" true true false 8 Double 0 0,First,#,BikeCentrelines,WghtCycMn,-1,-1;SafeIdxMn "SafeIdxMn" true true false 8 Double 0 0,First,#,BikeCentrelines,SafeIdxMn,-1,-1;AccIdxMn "AccIdxMn" true true false 8 Double 0 0,First,#,BikeCentrelines,AccIdxMn,-1,-1;BkShrIdxMn "BkShrIdxMn" true true false 8 Double 0 0,First,#,BikeCentrelines,BkShrIdxMn,-1,-1;BkPrkIdxMn "BkPrkIdxMn" true true false 8 Double 0 0,First,#,BikeCentrelines,BkPrkIdxMn,-1,-1;BkColIdxMn "BkColIdxMn" true true false 8 Double 0 0,First,#,BikeCentrelines,BkColIdxMn,-1,-1;BkThfIdxMn "BkThfIdxMn" true true false 8 Double 0 0,First,#,BikeCentrelines,BkThfIdxMn,-1,-1;StLtIdxMn "StLtIdxMn" true true false 8 Double 0 0,First,#,BikeCentrelines,StLtIdxMn,-1,-1;BkSafIdxMn "BkSafIdxMn" true true false 8 Double 0 0,First,#,BikeCentrelines,BkSafIdxMn,-1,-1;Shape_Length "Shape_Length" false true true 8 Double 0 0,First,#,BikeCentrelines,Shape_Length,-1,-1;Distance "Distance" true true false 8 Double 0 0,First,#,BikeCentrelines,Distance,-1,-1;Travel_Time__Min_ "Travel Time (Min)" true true false 512 Text 0 0,First,#,BikeCentrelines,Travel_Time__Min_,0,511;OBJECTID_First "OBJECTID_First" true true false 255 Long 0 0,First,#,C:\App Challenge 2026\ScriptTest4\ScriptTest4.gdb\BikeCentrelines,OBJECTID,-1,-1',
    sort_field=None
)

<Result 'C:\\App Challenge 2026\\ScriptTest5\\BikeCentrelines_Clean.shp'>